In [5]:
mortality_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/mortality_data_2000-2019.csv'
population_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/Population_par_commune.xlsx'

In [38]:
import pandas as pd
import numpy as np

#################                weekly mortality data divided by municipality (2000-2019)
week_mor_mun = pd.read_csv(mortality_url)
week_mor_mun['WEEK'] = week_mor_mun['YEAR_WEEK'].str.slice(start=6).astype('int') # get week number
week_mor_mun.rename(columns = {'N_MASK':'N_DEATHS'}, inplace = True)
del week_mor_mun['YEAR_WEEK']
week_mor_mun = week_mor_mun[week_mor_mun.COD != 'external'] # do not need data for external causes of death
week_mor_mun = week_mor_mun[week_mor_mun.ARRON != 58000] # data for Arrondissement de La Louvière is only available for 2019 and therefore has been removed from dataset
week_mor_mun.head()
#week_mor_mun.info()

,YEAR,COD,ARRON,N_DEATHS,WEEK
17,2000,natural,11000,31.0,1
18,2000,natural,12000,10.0,1
19,2000,natural,13000,14.0,1
20,2000,natural,21000,43.0,1
21,2000,natural,23000,14.0,1


In [9]:
##################               combine data sets from different excel sheets

# data with yearly population data divided by municipality
xls = pd.ExcelFile(population_url)

# create base dataframe
cols = ['municipality_code', 'municipality_name', 'male', 'female', 'total']
year_pop_mun = pd.DataFrame(columns=cols)
years = []
for i in range(21): # need poulation data for 21 years (years on both ends)
    df = pd.read_excel(xls, 'Population en '+str(2000+i), header=None, skiprows=4, nrows=646) # 2000-2019
    df.columns = cols
    length_of_df = df.loc[df['municipality_name'] == 'Viroinval'].index.tolist()[0] + 1 # get the last line of table
    years = years+[2000+i]*length_of_df
    year_pop_mun = pd.concat([year_pop_mun, df[:length_of_df]] , axis=0, ignore_index=True)
year_pop_mun['year'] = years
    
# # last dataframe is missing 8 municipalities
# year_pop_mun.drop(year_pop_mun.tail(4).index,inplace=True)
# year_pop_mun['year'] = years[:-8]

# convert population columns to int
convert_dict = {'male': int, 'female': int, 'total': int, 'municipality_code':int}
year_pop_mun = year_pop_mun.astype(convert_dict)

In [42]:
year_pop_mun

,municipality_code,municipality_name,male,female,total,year
0,1000,Belgique,5006014,5233071,10239085,2000
1,2000,Région flamande,2929510,3010741,5940251,2000
2,3000,Région wallonne,1618652,1720864,3339516,2000
3,4000,Région de Bruxelles-Capitale,457852,501466,959318,2000
4,10000,Province d'Anvers,810339,833633,1643972,2000
5,11000,Arrondissement d'Anvers,455850,476253,932103,2000
6,11001,Aartselaar,7158,7280,14438,2000
7,11002,Anvers,216580,229945,446525,2000
8,11004,Boechout,5990,5929,11919,2000
9,11005,Boom,7309,7748,15057,2000


In [12]:
##################                assign population to weekly death rate data (at week 1)
existing_arrons = week_mor_mun['ARRON'].value_counts().index
week_mor_mun['population_t'] = np.nan
exceptions = []
for year in range(21):
    for arron in existing_arrons:
        value = year_pop_mun[(year_pop_mun['year'] == (2000+year)) & (year_pop_mun['municipality_code'] == arron)]['total'].tolist()
        if len(value) == 1: # check if population value exists in year_pop_mun
            condition = (week_mor_mun['ARRON'] == arron) & (week_mor_mun['WEEK'] == 1) & (week_mor_mun['YEAR'] == (2000+year))
            if (condition).any(): # check if a row with week 1 exists in week_mor_mun
                week_mor_mun.loc[condition, 'population_t'] = value[0]
            else:
                dummy_df = pd.DataFrame({'YEAR': [2000+year], 'COD': ['natural'], 'ARRON': [arron], 'N_DEATHS': [0], 'WEEK':[1], 'population_t':  [value[0]]})
                week_mor_mun = pd.concat([week_mor_mun, dummy_df] , axis=0, ignore_index=True)
        else:
            exceptions.append((year, arron))

In [13]:
exceptions

[(19, 54000), (20, 54000)]

In [14]:
week_mor_mun['ARRON'].value_counts()

64000    1065
12000    1065
62000    1065
92000    1065
24000    1065
73000    1065
35000    1065
63000    1065
46000    1065
93000    1065
32000    1065
42000    1065
25000    1065
72000    1065
55000    1065
38000    1065
21000    1065
85000    1065
45000    1065
13000    1065
61000    1065
44000    1065
11000    1065
41000    1065
56000    1065
71000    1065
52000    1065
33000    1065
31000    1065
83000    1065
34000    1065
91000    1065
57000    1065
23000    1065
53000    1065
36000    1065
84000    1064
37000    1064
43000    1064
51000    1064
81000    1063
82000    1062
54000    1011
Name: ARRON, dtype: int64

In [15]:
# get number of weeks for each year
no_weeks = [week_mor_mun[(week_mor_mun['YEAR'] == (2000+i))]['WEEK'].max() for i in range(20)] # weeks for each year
for year in range(20): # for each year
    for arron in existing_arrons: # for each arron
        if arron == 54000 and (year == 18 or year == 19): # 54000 population not available for 2019
            continue
        condition = (week_mor_mun['ARRON'] == arron) & (week_mor_mun['WEEK'] == 1)
        prev_year_pop = week_mor_mun.loc[condition & (week_mor_mun['YEAR'] == (2000+year)), 'population_t'].tolist()[0]
        next_year_pop = week_mor_mun.loc[condition & (week_mor_mun['YEAR'] == (2000+year+1)), 'population_t'].tolist()[0]
        inter_values = np.linspace(prev_year_pop, next_year_pop, no_weeks[year], endpoint=False, dtype=int)[1:]
        for ix, k in enumerate(inter_values):
            week_mor_mun.loc[(week_mor_mun['ARRON'] == arron) & (week_mor_mun['YEAR'] == (2000+year)) & (week_mor_mun['WEEK'] == ix + 2),'population_t'] = k

In [16]:
print(arron, year)

54000 19


In [37]:
week_mor_mun.head(5)

,YEAR,COD,ARRON,N_DEATHS,WEEK,population_t
0,2000,natural,11000,31.0,1,932103.0
1,2000,natural,12000,10.0,1,305670.0
2,2000,natural,13000,14.0,1,406199.0
3,2000,natural,21000,43.0,1,959318.0
4,2000,natural,23000,14.0,1,558220.0


In [35]:
import os
os.getcwd()
os.chdir("C:\\Users\\baris\\Desktop\\Modern Data Analytics\\mda_project")

In [36]:
week_mor_mun.to_csv('out/weekly_mortality_with_linear_interpolated_population_data.csv', index=False)